In [ ]:
import os
os.environ['DOMINO_STARTING_USERNAME']

If you are using S3 based artifact store make sure you have AWS User configured who name is same
as the name of the variable `DOMINO_STARTING_USERNAME` above. Ask your adminstrator to provide you keys.

Before starting this workspace you should have the following environment variables set correctly in your
user profile:
- `AWS_ACCESS_KEY_ID`
- `AWS_SECRET_ACCESS_KEY`
- `AWS_DEFAULT_REGION`

If you are using a NFS (EFS) based artifact store, you do not need to do anything. Simply verify that the following folder exists `/artifacts/mnt/$DOMINO_STARTING_USERNAME`

In [ ]:
#Just some imports you will need
import mlflow
import jwt
import json
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [ ]:
print('MLFLOW_TRACKING_URI: ' + os.environ['MLFLOW_TRACKING_URI'])
print('MLFLOW_UI_URI: ' + os.environ['MLFLOW_UI_URI'])
print('MLFLOW_TRACKING_TOKEN: ' + os.environ['MLFLOW_TRACKING_TOKEN'])

In [ ]:
print('-----Decoded Value----')
print(jwt.decode(os.environ['MLFLOW_TRACKING_TOKEN'], "secret", algorithms=["HS256"]))
print('----------------------')
print('DOMINO_USER_API_KEY: ' + os.environ['DOMINO_USER_API_KEY'])
print('DOMINO_PROJECT_NAME: ' + os.environ['DOMINO_PROJECT_NAME'])
print('DOMINO_RUN_ID: ' + os.environ['DOMINO_RUN_ID'])

In [ ]:
#MFLOW Tracking URI which is set via environment variable MLFLOW_TRACKING_URI
print('MLFLOW_TRACKING_URI FROM MLFLOW LIBRARY: '+ (mlflow.get_tracking_uri()))
print('MLFLOW_TRACKING_URI FROM ENV VARIABLE : ' + os.environ['MLFLOW_TRACKING_URI'])

### Create MLFlow Client
We are ready to start using MLFLOW. Let us create an MLFlow Client. Note that we do not provide any parameters. It knows the Tracking URI and our Authentication Token from the environment variables created by the workspace environment.

In [ ]:
client = mlflow.tracking.MlflowClient()

### Create a new experiment or use an existing one

Provide a value for `myprefix` variable below which is unused to create a brand new 
experiment. Or simply leave it as it is. The experiment name will be the combination of 
- myprefix
- today's date
- user-name
- project-name

If the experiment with that name exists, you will be using it to create runs or a new experiment will be created and you will create runs into it

In [ ]:
myprefix=''

from datetime import datetime
now = datetime.now()
date_time_str = now.strftime("%m-%d-%Y")

experiment_name = date_time_str+'-'+'DEMO4'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
model_name = 'DEMO4'+'-' + os.environ['DOMINO_PROJECT_NAME']
if myprefix!='':
    experiment_name = myprefix + '-' + experiment_name

print(experiment_name)


In [ ]:
experiment = client.get_experiment_by_name(name=experiment_name)
if(experiment is None):
    print('Creating experiment ')
    client.create_experiment(name=experiment_name)
    experiment = client.get_experiment_by_name(name=experiment_name)

print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

In [ ]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

### Execute a run for the experiment


In [ ]:
# Read the wine-quality csv file from the URL
csv_url = (
     "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
     data = pd.read_csv(csv_url, sep=";")
except Exception as e:
     logger.exception(
          "Unable to download training & test CSV, check your internet connection. Error: %s", e
      )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]


my_log = "This is a test log"
with open("/tmp/test.txt", 'w') as f:
    f.write(my_log)
with open("/tmp/test.log", 'w') as f:
    f.write(my_log)

    
    
#run_tags={'mlflow.user':os.environ['DOMINO_STARTING_USERNAME']}
#Change user name
alpha = 0.7
l1_ratio = 0.6
with mlflow.start_run():
    print('Start Run')
    print('Alpha : ' + str(alpha))
    print('L1_Ratio : ' + str(l1_ratio))
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    print(mlflow.get_tracking_uri())
    print(tracking_url_type_store)
    # Model registry does not work with file store
    if tracking_url_type_store != "file":
      # Register the model
      # There are other ways to use the Model Registry, which depends on the use case,
      # please refer to the doc for more information:
      # https://mlflow.org/docs/latest/model-registry.html#api-workflow
      #mlflow.sklearn.log_model(lr, os.environ['DOMINO_STARTING_USERNAME']+"/model", registered_model_name="DEMO-ElasticnetWineModel-11")
      mlflow.sklearn.log_model(lr, "model", registered_model_name=model_name)
    else:        
      mlflow.sklearn.log_model(lr,  model_name)
      print(mlflow.get_artifact_uri())

Go to the MLFlow UI and view the following:
- Experiment
- Runs in the experiment
- Compare runs for performance
- View model and its versions (One per run)